In [1]:
%pip install pandas transformers torch nltk spacy pysbd scikit-learn rank-bm25 rouge-score numpy json5 rouge bert_score

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch
import os
from nltk.tokenize import sent_tokenize
import spacy
import pysbd
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import pandas as pd
import numpy as np
import json
import re
from rouge import Rouge
import concurrent.futures


c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
train_path = 'dataset_legal-pegasus/dataset/UK-Abs/train-data'
test_path = 'dataset_legal-pegasus/dataset/UK-Abs/test-data'

train_path_txt = train_path + '/judgement'
train_path_summary = train_path + '/summary'
test_path_txt = test_path + '/judgement'
test_path_summary = test_path + '/summary'

In [4]:
# Parcourir les fichiers dans le dossier de train et de test
train_files = os.listdir(train_path_txt)
test_files = os.listdir(test_path_txt)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Vérifier si le modèle est sur GPU
print(f"Le modèle est sur : {device}")


# Charger le modèle Legal-Pegasus et le tokenizer
model_name = "nsi319/legal-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Le modèle est sur : cuda


c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_locatio

In [5]:
def open_file(file_path, type):

    with open(file_path, 'r', encoding="utf-8") as f:
        if (type == "json"):
            return json.load(f)
        elif (type == "txt"):
            return f.read()

def split_text(text, tokenizer, max_length=1024):
    """Split the text into chunks that fit the model's input size"""
    input_ids = tokenizer.encode(text, return_tensors='pt', truncation=False)
    chunks = []
    for i in range(0, len(input_ids[0]), max_length):
        chunk = input_ids[:, i:i+max_length]
        chunks.append(chunk)
    return chunks

def summarize_large_text(text, model_name="legal-pegasus", min_length=150, max_length=250):
    """Summarize large texts by splitting them into chunks and summarizing each chunk"""

    if model_name == "legal-pegasus":
        tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
        model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")

        chunks = split_text(text, tokenizer)
        print(len(chunks))
        summaries = []

        for chunk in chunks[:2]:
            summary_ids = model.generate(chunk,
                                         num_beams=9,
                                         no_repeat_ngram_size=3,
                                         length_penalty=2.0,
                                         min_length=min_length,
                                         max_length=max_length,
                                         early_stopping=True)
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
            summaries.append(summary)

        # Combine all chunk summaries into one
        return ' '.join(summaries)

    else:
        return "Model not available"

def evaluation(text, ref):
    rouges = rouge_evaluations(text, ref)

def rouge_evaluations(text, ref):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(text, ref)

    return rouge_to_df(scores)

def rouge_to_df(scores):

    data = {
        'Metric': [],
        'Precision': [],
        'Recall': [],
        'F1-Score': []
    }

    for metric, score in scores.items():
        data['Metric'].append(metric)
        data['Precision'].append(score.precision)
        data['Recall'].append(score.recall)
        data['F1-Score'].append(score.fmeasure)

    return pd.DataFrame(data)

In [6]:
reference_judgement = open_file(train_path_txt + '/' + train_files[0], 'txt')

generated_summary = summarize_large_text(reference_judgement, model_name="legal-pegasus")

Token indices sequence length is longer than the specified maximum sequence length for this model (54024 > 1024). Running this sequence through the model will result in indexing errors


53


In [7]:
print(generated_summary)

On 13 December 2006 the appellant Mohammed al Ghabra, referred to in these proceedings as G, was informed that a direction had been made against him by HM Treasury under article 4 of the Terrorism (United Nations Measures) Order 2006 (SI 2006/2657). He was told that the Treasury had reasonable grounds for suspecting that he was, or might be, a person who facilitated the commission of acts of terrorism. The effect of the direction was to prohibit him from dealing with his funds and economic resources and to prevent anyone notified of the freeze from making funds, economic resources or financial services available to him or for his benefit. On 2 August 2007 the appellants Mohammed Jabar Ahmed, Mohammed Azmir Khan and Michael Marteen (formerly known as Mohammed Tunveer Ahmed) received letters in almost identical terms telling them that they had been designated under the TO by the Treasury. In September 2005 Hani El Sayed Sabaei Youssef (or Hani al Sebai) was also told that his name had be

In [8]:
# Fonction pour calculer les scores ROUGE
def compute_rouge_scores(reference_summary, generated_summary):
    print("flag 3")
    rouge = Rouge()

    # Calcul des scores ROUGE-1, ROUGE-2 et ROUGE-L
    scores = rouge.get_scores(generated_summary, reference_summary)
    print("flag3-1")


    # S'assurer que des scores ont été calculés avant de procéder
    if len(scores) == 0:
        return {"error": "No scores could be computed. Check your input summaries."}

    # Calcul de la moyenne des scores
    avg_scores = {
        'rouge-1': {
            'precision': sum([score['rouge-1']['p'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'recall': sum([score['rouge-1']['r'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'f1-score': sum([score['rouge-1']['f'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
        },
        'rouge-2': {
            'precision': sum([score['rouge-2']['p'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'recall': sum([score['rouge-2']['r'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'f1-score': sum([score['rouge-2']['f'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
        },
        'rouge-l': {
            'precision': sum([score['rouge-l']['p'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'recall': sum([score['rouge-l']['r'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'f1-score': sum([score['rouge-l']['f'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
        }
    }

    return avg_scores


In [9]:
reference_summary = open_file(train_path_summary + '/' + "uksc-2009-0018.txt", 'txt')

rouge_scores = compute_rouge_scores(reference_summary, generated_summary)

print("flag 4")

# Vérification et affichage des scores
if "error" in rouge_scores:
    print(rouge_scores["error"])
else:
    print("ROUGE-1: ", rouge_scores['rouge-1'])
    print("ROUGE-2: ", rouge_scores['rouge-2'])
    print("ROUGE-L: ", rouge_scores['rouge-l'])

flag 3
flag3-1
flag 4
ROUGE-1:  {'precision': 0.3607843137254902, 'recall': 0.20175438596491227, 'f1-score': 0.25879043140522356}
ROUGE-2:  {'precision': 0.10869565217391304, 'recall': 0.048283261802575105, 'f1-score': 0.06686478028733008}
ROUGE-L:  {'precision': 0.3176470588235294, 'recall': 0.17763157894736842, 'f1-score': 0.22784809666542052}


In [10]:
print(len(generated_summary))
print(len(reference_summary))

2629
6922


In [11]:
from bert_score import score

# Calculer le score BERTScore
P, R, F1 = score([generated_summary], [reference_summary], lang="en", verbose=True)

# Afficher les scores de Précision, Rappel et F1
print(f"Précision (P): {P.mean().item():.4f}")
print(f"Rappel (R): {R.mean().item():.4f}")
print(f"F1-Score: {F1.mean().item():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 8.34 seconds, 0.12 sentences/sec
Précision (P): 0.8100
Rappel (R): 0.8229
F1-Score: 0.8164
